In [ ]:
import numpy as np
import pandas as pd
import json
import os
from datetime import datetime
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

### Execution parameters

In [ ]:
SEED = 10
limit_rows = (1)*1000

In [ ]:
def get_time():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

### Read individual data sets

In [ ]:
base_folder = 'D:/Workspace/IoT-23 Dataset/iot_23_datasets_small/opt/Malware-Project/BigDataset/IoTScenarios/'
sub_folders = filter(lambda x : ('Honeypot' not in x and 'bro' in x), [x[0] for x in os.walk(base_folder)])
base_filename = 'conn.log.labeled'

In [ ]:
data_frames = []

for folder in sub_folders:  
    
    full_filename = folder.replace('\\','/') + '/' + base_filename
    
    print(f"{get_time()} Processing folder '{folder}' started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")
    
    df = pd.read_table(filepath_or_buffer=full_filename, skiprows=8, nrows=limit_rows)
    
    df.columns = [
        'ts',
        'uid',
        'id.orig_h',
        'id.orig_p',
        'id.resp_h',
        'id.resp_p',
        'proto',
        'service',
        'duration',
        'orig_bytes',
        'resp_bytes',
        'conn_state',
        'local_orig',
        'local_resp',
        'missed_bytes',
        'history',
        'orig_pkts',
        'orig_ip_bytes',
        'resp_pkts',
        'resp_ip_bytes',
        'label'
    ]
    
    df.drop(columns=['ts','uid','service','local_orig','local_resp','history','id.orig_h','id.resp_h'], inplace=True)
        
    df.drop(df.tail(1).index, inplace=True)
    
    print(f'{get_time()} Duplicated rows (before removal):\t{df.duplicated().sum()}')
    df.drop_duplicates(inplace=True)
    print(f'{get_time()} Duplicated rows (after removal):\t{df.duplicated().sum()}')
    
    data_frames.append(df)
        
    print(f"{get_time()} Processing finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}; DF shape: {df.shape}.\n")

### Concatenate data sets

In [ ]:
df_c = pd.concat(data_frames)

In [ ]:
df_c

### Standardize labels

In [ ]:
df_c['label'].value_counts()

In [ ]:
def replace_label(src,dest):
    df_c.loc[(df_c.label == src), 'label'] = dest

In [ ]:
replace_label('-   Malicious   Attack',                                  'Attack')
replace_label('(empty)   Malicious   Attack',                            'Attack')
replace_label('-   Benign   -',                                          'Benign')
replace_label('(empty)   Benign   -',                                    'Benign')
replace_label('COLnd035cNITygYHp3   Benign   -',                         'Benign')
replace_label('-   Malicious   C&C',                                     'C&C')
replace_label('(empty)   Malicious   C&C',                               'C&C')
replace_label('-   Malicious   C&C-FileDownload',                        'C&C-FileDownload')
replace_label('-   Malicious   C&C-HeartBeat',                           'C&C-HeartBeat')
replace_label('(empty)   Malicious   C&C-HeartBeat',                     'C&C-HeartBeat')
replace_label('-   Malicious   C&C-HeartBeat-Attack',                    'C&C-HeartBeat-Attack')
replace_label('-   Malicious   C&C-HeartBeat-FileDownload',              'C&C-HeartBeat-FileDownload')
replace_label('-   Malicious   C&C-HeartBeat-PartOfAHorizontalPortScan', 'C&C-HeartBeat-PartOfAHorizontalPortScan')
replace_label('-   Malicious   C&C-Mirai',                               'C&C-Mirai')
replace_label('-   Malicious   C&C-PartOfAHorizontalPortScan',           'C&C-PartOfAHorizontalPortScan')
replace_label('-   Malicious   C&C-Torii',                               'C&C-Torii')
replace_label('-   Malicious   DDoS',                                    'DDoS')
replace_label('(empty)   Malicious   DDoS',                              'DDoS')
replace_label('-   Malicious   FileDownload',                            'FileDownload')
replace_label('-   Malicious   Okiru',                                   'Okiru')
replace_label('(empty)   Malicious   Okiru',                             'Okiru')
replace_label('-   Malicious   Okiru-Attack',                            'Okiru-Attack')
replace_label('-   Malicious   PartOfAHorizontalPortScan',               'PartOfAHorizontalPortScan')
replace_label('(empty)   Malicious   PartOfAHorizontalPortScan',         'PartOfAHorizontalPortScan')
replace_label('-   Malicious   PartOfAHorizontalPortScan-Attack',        'PartOfAHorizontalPortScan-Attack')

In [ ]:
df_c['label'].value_counts()

### Remove less frequent classes

In [ ]:
vc = df_c['label'].value_counts(normalize=True)
print(f'**** Value counts (before) ****\n{vc}')
threshold = vc.quantile(0.75)
print(f'\n**** Dropping rows with less than {threshold:g} occurrences ****\n')
df_c = df_c[df_c['label'].isin(vc.index[vc.gt(threshold)])]
vc = df_c['label'].value_counts(normalize=True)
print(f'**** Value counts (after) ****\n{vc}')

### Replace missing values

In [ ]:
df_c.loc[(df_c.duration == '-'), 'duration'] = 0.0
df_c.loc[(df_c.orig_bytes == '-'), 'orig_bytes'] = 0
df_c.loc[(df_c.resp_bytes == '-'), 'resp_bytes'] = 0

### Remove NaN values

In [ ]:
print('NaN values (before removal):','\n',df_c.isna().sum())
df_c = df_c.dropna()
print('\nNaN values (after removal):','\n',df_c.isna().sum())

### Remove duplicate rows

In [ ]:
current_cols = [x for x in df_c.columns]
print('Duplicated rows (before removal):\t',df_c.duplicated().sum())
df_c['count'] = 1
df_c = df_c.groupby(current_cols)['count'].count().reset_index().drop_duplicates()
print('Duplicated rows (after removal):\t',df_c.duplicated().sum())

### One-hot-encode categorical features

In [ ]:
to_be_encoded = ['proto','conn_state']#,'id.orig_p','id.resp_p']
for col in to_be_encoded:
    df_c = pd.get_dummies(df_c, columns=[col])
    print(f'Column \'{col}\' successfully one-hot-encoded; new DF shape: {df_c.shape}.')

### Reorder columns to [..., count, label]

In [ ]:
final_cols = [x for x in df_c.columns.values if x != 'label' and x != 'count']
final_cols.append('count')
final_cols.append('label')
print(final_cols,type(final_cols))
df_c = df_c.reindex(columns=final_cols)

### Define data types

In [ ]:
df_c = df_c.astype({
    'id.orig_p'     : 'uint64',
    'id.resp_p'     : 'uint64',
    'duration'      : float,
    'orig_bytes'    : 'uint64',
    'resp_bytes'    : 'uint64',
    'missed_bytes'  : 'uint64',
    'orig_pkts'     : 'uint64',
    'orig_ip_bytes' : 'uint64',
    'resp_pkts'     : 'uint64',
    'resp_ip_bytes' : 'uint64',
    'label'         : 'category'
}).infer_objects()

In [ ]:
df_c

In [ ]:
print(json.dumps(dict(df_c.dtypes), indent=4, default=str))

### Split data intro train/test subsets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_c.drop(labels=['label'], axis=1),
                                                    df_c['label'],
                                                    test_size=0.2,
                                                    random_state=SEED)

### Select relevant features

In [ ]:
from sklearn.feature_selection import VarianceThreshold

constant_filter = VarianceThreshold(threshold=0.01)
constant_filter.fit(X_train)

print('\nNon quasi-constant columns:')
non_constant_columns = [column for column in X_train.columns if column not in X_train.columns[constant_filter.get_support()]]
print(non_constant_columns)

print('\nQuasi-constant columns:')
constant_columns = [column for column in X_train.columns if column in X_train.columns[constant_filter.get_support()]]
print(constant_columns)

X_train = constant_filter.transform(X_train)
X_test = constant_filter.transform(X_test)

### Scale data to [0,1] range

In [ ]:
from sklearn.preprocessing import MinMaxScaler

range_scaler = MinMaxScaler()
range_scaler.fit(X_train)

X_train = range_scaler.transform(X_train)
X_test = range_scaler.transform(X_test)

### Persist train and test data subsets

In [ ]:
def persist_csv(df,name):
    if limit_rows is None:        
        pd.DataFrame(df).to_csv(f'sklearn/iot23_combined_{name}.csv',
                                float_format='%g',
                                header=None,
                                index=None)
    else:
        pd.DataFrame(df).round(6).to_csv(f'sklearn/iot23_combined_{int(limit_rows/1000)}k_{name}.csv',
                                float_format='%g',
                                header=None,
                                index=None)

In [ ]:
persist_csv(X_train,'X_train')
persist_csv(y_train,'y_train')
print('X_train',X_train.shape,'\ny_train',y_train.shape)

persist_csv(X_test,'X_test')
persist_csv(y_test,'y_test')
print('X_test',X_test.shape,'\ny_test',y_test.shape)